In [1]:
# Importing libraries and packages:

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

Using TensorFlow backend.


In [4]:
from keras import backend
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [22]:
import pandas as pd
 
train_df = pd.read_csv('sales_train_v2.csv')
train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [23]:
#Group monthly item_cnt amount
temp = train_df.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].sum().reset_index()
temp2 = train_df.groupby(['date_block_num','shop_id','item_id'])['item_price'].mean().reset_index()
train_df = pd.merge(temp, temp2, how='inner', on=['date_block_num', 'shop_id','item_id'])
temp = []
temp2 = []
train_df

,date_block_num,shop_id,item_id,item_cnt_day,item_price
0,0,0,32,6.0,221.0
1,0,0,33,3.0,347.0
2,0,0,35,1.0,247.0
3,0,0,43,1.0,221.0
4,0,0,51,2.0,128.5
5,0,0,61,1.0,195.0
6,0,0,75,1.0,76.0
7,0,0,88,1.0,76.0
8,0,0,95,1.0,193.0
9,0,0,96,1.0,70.0


In [24]:
test_df = pd.read_csv('test.csv')
test_df.shape

(214200, 3)

In [25]:
X_train = train_df
X_test = train_df[train_df['date_block_num'].isin([33])]

In [26]:
temp = train_df[['shop_id','item_id','item_price']].groupby(['shop_id','item_id'])['item_price'].min().reset_index()
test_df = pd.merge(test_df, temp, how='left', on=['shop_id','item_id'])
test_df.shape

(214200, 4)

In [27]:
X_train["date_block_num"] = X_train["date_block_num"].apply(lambda x: x%12)
# X_test["date_block_num"] = X_test["date_block_num"].apply(lambda x: x%12)

In [28]:
feature_cols = X_train.columns.values

X = X_train[feature_cols]  

y = X_train['item_cnt_day']

X = X.drop(['item_cnt_day','item_id'], axis=1)

In [30]:
X = pd.get_dummies(X,columns=['date_block_num','shop_id'])
scalerX = MinMaxScaler(copy=True, feature_range=(0, 1))
X = pd.DataFrame(data = scalerX.fit_transform(X), columns = X.columns.values)

c:\users\daniel chang\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Keras

In [18]:
model = Sequential()
model.add(Dense(2, input_dim=73, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='mse', optimizer='adam', metrics=[rmse])
model.fit(X_train, y_train, validation_split=0.2, batch_size=128, epochs=4, verbose=1)

W0919 01:07:32.423914 11776 deprecation_wrapper.py:119] From C:\Users\Daniel Chang\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0919 01:07:32.460815 11776 deprecation_wrapper.py:119] From C:\Users\Daniel Chang\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 1287299 samples, validate on 321825 samples
Epoch 1/4
1287299/1287299 [==============================] - 26s 20us/step - loss: 70.0347 - rmse: 1.3037 - val_loss: 101.9922 - val_rmse: 1.1490
Epoch 2/4
1287299/1287299 [==============================] - 25s 19us/step - loss: 70.0347 - rmse: 1.3037 - val_loss: 101.9922 - val_rmse: 1.1490
Epoch 3/4
1287299/1287299 [==============================] - 25s 20us/step - loss: 70.0347 - rmse: 1.3037 - val_loss: 101.9922 - val_rmse: 1.1490
Epoch 4/4
1287299/1287299 [==============================] - 25s 20us/step - loss: 70.0347 - rmse: 1.3037 - val_loss: 101.9922 - val_rmse: 1.1490


# Linear Regression

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                                    random_state=2)
my_linreg = LinearRegression()

my_linreg.fit(X_train, y_train)

y_prediction = my_linreg.predict(X_test)
# Calculating "Mean Square Error" (MSE):
mse = metrics.mean_squared_error(y_test, y_prediction)

# Using numpy sqrt function to take the square root and calculate "Root Mean Square Error" (RMSE)
rmse = np.sqrt(mse)
rmse

8.684918690794719

# Output csv file 

In [32]:
test_df['date_block_num'] = 10
test_df.head()

,ID,shop_id,item_id,item_price,date_block_num
0,0,5,5037,749.5,10
1,1,5,5320,NaN,10
2,2,5,5233,599.0,10
3,3,5,5232,599.0,10
4,4,5,5268,NaN,10


# New Items in 2015 Nov

In [33]:
temp = test_df[test_df.isna().any(axis=1)]
temp

,ID,shop_id,item_id,item_price,date_block_num
1,1,5,5320,NaN,10
4,4,5,5268,NaN,10
9,9,5,5003,NaN,10
16,16,5,4874,NaN,10
21,21,5,5002,NaN,10
25,25,5,5907,NaN,10
26,26,5,5908,NaN,10
31,31,5,5668,NaN,10
45,45,5,5826,NaN,10
64,64,5,3538,NaN,10


In [34]:
test_df = test_df.fillna(method='ffill')
test_df

,ID,shop_id,item_id,item_price,date_block_num
0,0,5,5037,749.500000,10
1,1,5,5320,749.500000,10
2,2,5,5233,599.000000,10
3,3,5,5232,599.000000,10
4,4,5,5268,599.000000,10
5,5,5,5039,749.500000,10
6,6,5,5041,3832.333333,10
7,7,5,5046,299.000000,10
8,8,5,5319,299.000000,10
9,9,5,5003,299.000000,10


In [35]:
feature_cols = X.columns.values

# use the above list to select the features from the original DataFrame
test = test_df[feature_cols]
test = test.drop(['ID','item_id'], axis=1)
test.head()

,shop_id,item_price,date_block_num
0,5,749.5,10
1,5,749.5,10
2,5,599.0,10
3,5,599.0,10
4,5,599.0,10


In [43]:
# test = pd.get_dummies(test,columns=['date_block_num','shop_id'])
test['date_block_num_0']=0
test['date_block_num_1']=0
test['date_block_num_2']=0
test['date_block_num_3']=0
test['date_block_num_4']=0
test['date_block_num_5']=0
test['date_block_num_6']=0
test['date_block_num_7']=0
test['date_block_num_8']=0
test['date_block_num_9']=0
test['date_block_num_11']=0
test['shop_id_0']=0
test['shop_id_1']=0
test['shop_id_8']=0
test['shop_id_9']=0
test['shop_id_11']=0
test['shop_id_13']=0
test['shop_id_17']=0
test['shop_id_20']=0
test['shop_id_23']=0
test['shop_id_27']=0
test['shop_id_29']=0
test['shop_id_30']=0
test['shop_id_32']=0
test['shop_id_33']=0
test['shop_id_40']=0
test['shop_id_43']=0
test['shop_id_51']=0
test['shop_id_54']=0

In [44]:
test = pd.DataFrame(data = scalerX.transform(test), columns = test.columns.values)
test.head()

,item_price,date_block_num_10,shop_id_2,shop_id_3,shop_id_4,shop_id_5,shop_id_6,shop_id_7,shop_id_10,shop_id_12,...,shop_id_23,shop_id_27,shop_id_29,shop_id_30,shop_id_32,shop_id_33,shop_id_40,shop_id_43,shop_id_51,shop_id_54
0,0.002433,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.002433,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.001945,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.001945,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.001945,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
y_predict = my_linreg.predict(test)
y_predict.shape

(214200,)

In [47]:
raw_data = {'ID':test_df['ID'],
            'item_cnt_month':y_predict.flatten()}
df = pd.DataFrame(raw_data, columns = ['ID', 'item_cnt_month'])
df

,ID,item_cnt_month
0,0,-8.286709e+09
1,1,-8.286709e+09
2,2,-8.286709e+09
3,3,-8.286709e+09
4,4,-8.286709e+09
5,5,-8.286709e+09
6,6,-8.286709e+09
7,7,-8.286709e+09
8,8,-8.286709e+09
9,9,-8.286709e+09


In [51]:
df['item_cnt_month'] = df['item_cnt_month'].apply(lambda x: 0 if x<0 else x)
df

,ID,item_cnt_month
0,0,0.000000
1,1,0.000000
2,2,0.000000
3,3,0.000000
4,4,0.000000
5,5,0.000000
6,6,0.000000
7,7,0.000000
8,8,0.000000
9,9,0.000000


In [62]:
df['item_cnt_month'] = df['item_cnt_month'].where(~df['ID'].isin(temp['ID']), 0)
df

,ID,item_cnt_month
0,0,1.0
1,1,0.5
2,2,1.0
3,3,1.0
4,4,0.5
5,5,1.0
6,6,1.0
7,7,1.0
8,8,1.0
9,9,0.5


In [52]:
# raw_data = {'ID':test_df['ID'],
#             'item_cnt_month':y_predict.flatten()}
# df = pd.DataFrame(raw_data, columns = ['ID', 'item_cnt_month'])
df.to_csv('Result.csv',index=0)

In [53]:
result_df = pd.read_csv('Result.csv')
result_df['item_cnt_month'].unique()

array([0.        , 2.02614021, 1.96497345, ..., 2.05104828, 2.03862   ,
       2.02305031])